practical 2 : Implementation of S-AES

In [7]:
# 4-bit S-box and inverse S-box
SBOX = [0x9, 0x4, 0xA, 0xB,
        0xD, 0x1, 0x8, 0x5,
        0x6, 0x2, 0x0, 0x3,
        0xC, 0xE, 0xF, 0x7]

SBOX_INV = [0xA, 0x5, 0x9, 0xB,
            0x1, 0x7, 0x8, 0xF,
            0x6, 0x0, 0x2, 0x3,
            0xC, 0x4, 0xD, 0xE]

# Multiply in GF(2^4)
def mult(a, b):
    p = 0
    for _ in range(4):
        if b & 1:
            p ^= a
        carry = a & 0x8
        a = (a << 1) & 0xF
        if carry:
            a ^= 0x3  # modulo x⁴ + x + 1 (0b11)
        b >>= 1
    return p & 0xF

def sub_nibbles(s):
    result = 0
    for i in range(4):
        nibble = (s >> (4 * (3 - i))) & 0xF
        result |= (SBOX[nibble] << (4 * (3 - i)))
    return result

def shift_rows(s):
    # Swap the lower nibbles (simulate 2x2 matrix row shift)
    upper = (s & 0xF000) | (s & 0x0F00)
    lower = (s & 0x00F0) << 4 | (s & 0x000F) >> 4
    return upper | lower

def mix_columns(s):
    # Convert to 2x2 matrix form and apply MixColumns transformation
    s0 = (s >> 12) & 0xF
    s1 = (s >> 8) & 0xF
    s2 = (s >> 4) & 0xF
    s3 = s & 0xF

    m0 = mult(1, s0) ^ mult(4, s2)
    m1 = mult(1, s1) ^ mult(4, s3)
    m2 = mult(4, s0) ^ mult(1, s2)
    m3 = mult(4, s1) ^ mult(1, s3)

    return (m0 << 12) | (m1 << 8) | (m2 << 4) | m3

def add_round_key(s, k):
    return s ^ k

def key_expansion(key):
    RCON1 = 0x80
    RCON2 = 0x30

    w = [0] * 6
    w[0] = key >> 8
    w[1] = key & 0xFF
    w[2] = w[0] ^ RCON1 ^ sub_nibbles(w[1])
    w[3] = w[2] ^ w[1]
    w[4] = w[2] ^ RCON2 ^ sub_nibbles(w[3])
    w[5] = w[4] ^ w[3]
    return [(w[0] << 8) | w[1], (w[2] << 8) | w[3], (w[4] << 8) | w[5]]

def s_aes_encrypt(plaintext, key):
    round_keys = key_expansion(key)

    # Initial round key
    state = add_round_key(plaintext, round_keys[0])

    # Round 1
    state = sub_nibbles(state)
    state = shift_rows(state)
    state = mix_columns(state)
    state = add_round_key(state, round_keys[1])

    # Round 2 (Final round)
    state = sub_nibbles(state)
    state = shift_rows(state)
    state = add_round_key(state, round_keys[2])

    return state

# 🔽 Take user input for plaintext and key
plaintext_hex = input("Enter 16-bit plaintext in hex (e.g., D728): ")
key_hex = input("Enter 16-bit key in hex (e.g., 4AF5): ")

try:
    # Convert from hex string to integer
    plaintext = int(plaintext_hex, 16)
    key = int(key_hex, 16)

    # Encrypt and display result
    ciphertext = s_aes_encrypt(plaintext, key)
    print("Encrypted Ciphertext (hex):", hex(ciphertext).upper())

except ValueError:
    print("Invalid hex input. Please enter valid 4-digit hex values (0–FFFF).")


Enter 16-bit plaintext in hex (e.g., D728):  D728
Enter 16-bit key in hex (e.g., 4AF5):  4AF5


Encrypted Ciphertext (hex): 0XBB5B1A
